In [1]:
from mpl_toolkits.basemap import Basemap
from multiprocessing import Process,Array,Value
from collections import defaultdict
from scipy.spatial import distance as dst
import matplotlib.pyplot as plt
import shapely.geometry as sp
import numpy as np
import pandas as pd
import pickle as pk
import datetime as dt
import threading
import itertools
import math
import time
import json
import copy

In [4]:
flights=pd.read_csv("Outputs/cleanFlightHistory.csv",low_memory=False)
Tracks=pd.read_csv("MetricFiles/DayWiseInput/2013-08-16_tracks.csv")
m = pk.load(open("Outputs/M_ConversionMetric.pkl", "rb"))
speeds = pk.load(open("Outputs/SpeedDict.pkl", "rb"))
SectorGraph = pk.load(open("Outputs/SectorGraph.pkl", "rb"))
Centroids = pk.load(open("Outputs/CentroidDict.pkl", "rb"))
ConvexHulls = pk.load(open("Outputs/ConvexDict.pkl", "rb"))
airportSector=pk.load(open("Outputs/airportSectorDict.pkl", "rb"))

In [18]:
flightID=306048464

In [19]:
speedinKnots=speeds[flightID]
flight=flights[flights['id']==flightID]
start=airportSector[flight['departure_airport_icao_code'].values[0]]
end=airportSector[flight['arrival_airport_icao_code'].values[0]]
print(f"Input to GA:\n{start},{end},0,{speedinKnots}")

realDepart=dt.datetime.fromisoformat(str(flight['actual_runway_departure'].values[0]))
realArrival=dt.datetime.fromisoformat(str(flight['actual_runway_arrival'].values[0]))
flights[flights['id']==flightID]

Input to GA:
775,159,0,433.0


,id,departure_airport_icao_code,arrival_airport_icao_code,scheduled_runway_departure,actual_runway_departure,scheduled_runway_arrival,actual_runway_arrival
46330,306048464,KMCO,KLAS,2013-08-16 00:00:00,2013-08-16 00:04:00,2013-08-16 04:21:00,2013-08-16 04:28:00


In [20]:
%matplotlib qt
fig = plt.figure(pk.load(open("Outputs/Simulator.pkl","rb")))

In [21]:
pathFromGA=[989,980,983,998,990]

In [22]:
singleFlightTrack=Tracks[Tracks['flighthistory_id']==flightID]
singleFlightTrack=singleFlightTrack.sort_values('received')
lat=singleFlightTrack['latitude_degrees'].values
lon=singleFlightTrack['longitude_degrees'].values
realX,realY=m(lon,lat)
PathLenGA=0
PathLenReal=0
for pt in range(1,len(realX)):
     PathLenReal+=dst.euclidean((realX[pt-1],realY[pt-1]),(realX[pt],realY[pt]))
# mpm_speed=(speedinKnots/1.944)*60
GAx=[Centroids[pathFromGA[0]][0]]
GAy=[Centroids[pathFromGA[0]][1]]
for Hull in range(len(pathFromGA)-1):
    X=[]
    Y=[]
    Plot=ConvexHulls[pathFromGA[Hull]]
    Plot.append(ConvexHulls[pathFromGA[Hull]][0])
    for pt in Plot:
        X.append(pt[0])
        Y.append(pt[1])
    line1 = sp.LineString(list(Plot))
    for neigh in SectorGraph.neighbors(pathFromGA[Hull]):
        if(neigh==pathFromGA[Hull+1]):
            line2 = sp.LineString([Centroids[pathFromGA[Hull]], Centroids[neigh]])
            ABC=line1.intersection(line2)
            PathLenGA+=dst.euclidean((GAx[-1],GAy[-1]),(ABC.x,ABC.y))
            GAx.append(ABC.x)
            GAy.append(ABC.y)
PathLenGA+=dst.euclidean((GAx[-1],GAy[-1]),Centroids[pathFromGA[-1]])
GAx.append(Centroids[pathFromGA[-1]][0])
GAy.append(Centroids[pathFromGA[-1]][1])

In [23]:
plt.plot(realX,realY,linewidth=20,c='b',label="Actual Path")
# plt.plot(GAx,GAy,linewidth=20,c='g',label="GA Path")
plt.legend(loc="upper left",prop={'size': 100})
fig.canvas.draw()

In [116]:
import time
dfFlights=pd.read_csv("Outputs/cleanFlightHistory.csv")
ConvexDict = pk.load(open("Outputs/ConvexDict.pkl", "rb"))
dayDf=dfFlights[dfFlights['scheduled_runway_departure'].str.contains("2013-08-16")].copy()
dayDf.sort_values(by=["scheduled_runway_departure"], inplace = True)
dayDf=dayDf[:500]
stlol=time.time()
ProperPath=0
BadPath=0
NoPath=[]
HalfPath=[]
FullPath=[]
for index,flight in dayDf.iterrows():
    start=airportSector[flight['departure_airport_icao_code']]
    end=airportSector[flight['arrival_airport_icao_code']]
    if(start==end):
        continue
    singleFlightTrack=Tracks[Tracks['flighthistory_id']==flight['id']]
    singleFlightTrack=singleFlightTrack.sort_values('received')
    lat=singleFlightTrack['latitude_degrees'].values
    lon=singleFlightTrack['longitude_degrees'].values
    realX,realY=m(lon,lat)
    if(len(realX)==0):
        NoPath.append(flight['id'])
    else:
        StartPoint=(realX[0],realY[0])
        EndPoint=(realX[-1],realY[-1])
        if(not ((sp.Polygon(ConvexDict[start]).contains(sp.Point(StartPoint))) and (sp.Polygon(ConvexDict[end]).contains(sp.Point(EndPoint))))):
            HalfPath.append(flight['id'])
        else:
            FullPath.append(flight['id'])
enlol=time.time()
print(enlol-stlol)

43.76874017715454


In [118]:
FullPath

[306043857,
 306032763,
 306386689,
 306062641,
 306386460,
 306020239,
 306049726,
 306386664,
 306020280,
 306011591,
 306072605,
 306050965,
 306022596,
 306015773,
 306020071,
 306020366,
 306027806,
 306386666,
 306003307,
 306011989,
 306058850,
 306049714,
 306024070,
 305998957,
 306033418,
 306049035,
 306058386,
 306058843,
 306017890,
 306011477,
 306022466,
 306011214,
 306019913,
 305997446,
 306058102,
 306048394,
 306015693,
 306290201,
 306070058,
 306064127,
 306049769,
 306014514,
 306034438,
 306053374,
 306020193,
 306020168,
 306386483,
 306073087,
 306012060,
 306073168,
 305998255,
 306048998,
 306051345,
 306010929,
 306042573,
 306011762,
 306023030,
 306032967,
 306032965,
 306078541,
 306011347,
 306062776,
 306033419,
 306014555,
 306063029,
 306019998,
 306016627,
 306058255,
 306041442,
 306062996,
 306042308,
 306067010,
 306062733,
 306011705,
 306011859,
 306033354,
 306033124,
 306069846,
 305998980,
 306386696,
 306386701,
 306017877,
 306072689,
 306

In [121]:
speedDictFile=open("Outputs/FullPath.pkl","wb")
pk.dump(FullPath,speedDictFile)
speedDictFile.close()

In [29]:
maxiD=0
maxD=0
arr=[]
for i in FullPath:
    singleFlightTrack=Tracks[Tracks['flighthistory_id']==i]
    singleFlightTrack=singleFlightTrack.sort_values('received')
    lat=singleFlightTrack['latitude_degrees'].values
    lon=singleFlightTrack['longitude_degrees'].values
    PathLenReal=0
    realX,realY=m(lon,lat)
    for pt in range(1,len(realX)):
         PathLenReal+=dst.euclidean((realX[pt-1],realY[pt-1]),(realX[pt],realY[pt]))
    arr.append((i,PathLenReal))

In [33]:
arr = sorted(arr,key=lambda x:x[1])

In [90]:
arr[-10]

(306020193, 3753028.476356991)

In [119]:
print(len(NoPath))
print(len(HalfPath))
print(len(FullPath))

60
94
346


In [102]:
singleFlightTrack=Tracks[Tracks['flighthistory_id']==306020193]
g=list(singleFlightTrack['ground_speed'])

In [103]:
print(g)

[157, 253, 271, 0, 273, 306, 344, 359, 371, 378, 371, 394, 376, 395, 395, 399, 411, 418, 425, 431, 442, 437, 437, 433, 423, 419, 410, 411, 412, 407, 397, 398, 404, 409, 410, 416, 419, 421, 425, 423, 424, 418, 417, 424, 423, 422, 426, 423, 424, 424, 422, 423, 421, 422, 419, 424, 422, 419, 421, 421, 420, 423, 422, 423, 425, 426, 429, 427, 427, 425, 428, 427, 429, 430, 431, 431, 431, 432, 432, 421, 425, 422, 422, 424, 424, 419, 422, 426, 423, 419, 418, 417, 419, 416, 418, 414, 417, 409, 405, 415, 414, 415, 417, 416, 423, 424, 427, 427, 434, 434, 435, 436, 438, 438, 438, 438, 438, 438, 439, 439, 440, 439, 441, 440, 442, 439, 440, 441, 439, 441, 438, 439, 441, 440, 445, 443, 444, 445, 444, 445, 445, 444, 445, 446, 445, 446, 446, 446, 442, 440, 434, 436, 433, 434, 438, 443, 442, 442, 442, 438, 439, 433, 437, 431, 434, 427, 431, 424, 427, 424, 426, 425, 424, 425, 424, 427, 426, 429, 429, 435, 432, 437, 435, 440, 436, 440, 439, 442, 440, 438, 439, 442, 439, 438, 439, 434, 438, 441, 438, 437, 4

In [104]:
import numpy as np
a=np.array(g)

In [105]:
q1=len(g)//4
q3=3*(len(g))//4

In [106]:
su=0
c=0
for i in range(q1,q3+1):
    c+=1
    su+=g[i]
print(su/c)

432.7098445595855


In [109]:
x=g[q1:q3+1]
np.median(np.array(x))

434.0

In [107]:
max(g)

451

In [108]:
sum(g)/len(g)

409.09090909090907